# Doc2Vec: Como funciona

`Doc2Vec` é uma ferramenta para representação de documentos em forma de vetores, sendo uma extensão do `Word2Vec`.

Para fins didáticos, vou utilizar um dataset de *train* e *test* de **Análise de Sentimentos: Emoção no texto** (*tradução livre*) disponibilizado no site [Kaggle](https://www.kaggle.com/c/sa-emotions/data)

**Dicionário dos dados**
- **id:** Id do post
- **sentiment:** Emoção do texto
- **content:** texto

## Importando as bibliotecas

In [1]:
# importando para manipulação dos dados
import pandas as pd

# importando para NLP
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
nltk.download('stopwords')

# importando para Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# importando os dados
df_train = pd.read_csv('data/train_data.csv')

# verificando as primeiras entradas
df_train.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [3]:
# checando as dimensões do dataset
print(f'Dados de treino: {df_train.shape}')

Dados de treino: (30000, 2)


In [4]:
# verificando por quantidade as emoções classificadas
df_train['sentiment'].value_counts()

worry         7433
neutral       6340
sadness       4828
happiness     2986
love          2068
surprise      1613
hate          1187
fun           1088
relief        1021
empty          659
enthusiasm     522
boredom        157
anger           98
Name: sentiment, dtype: int64

In [5]:
# olhando a quantidade de elementos que compõe os posts
df_train['lenght_content'] = df_train['content'].apply(len)
df_train.head()

,sentiment,content,lenght_content
0,empty,@tiffanylue i know i was listenin to bad habi...,92
1,sadness,Layin n bed with a headache ughhhh...waitin o...,60
2,sadness,Funeral ceremony...gloomy friday...,35
3,enthusiasm,wants to hang out with friends SOON!,36
4,neutral,@dannycastillo We want to trade with someone w...,86


In [6]:
# criando uma lista para testar as classificações
shortlist = ['neutral', 'happiness', 'worry']

# criando um novo conjunto de dados menor
df_amostra = df_train[df_train['sentiment'].isin(shortlist)]

# checando a nova dimensão
df_amostra.shape

(16759, 3)

## Processamento do texto

Tweets são diferentes uns dos outros, portanto devemos considerar:
- Remover menções "@nome" e talvez url?
- Vamos utilizar *TweetTokenizer* ao invés de *regex*
- Stopwords, caracteres numericos, como feito normalmente

In [7]:
# instanciando o TweetTokenizer
tweeter = TweetTokenizer(strip_handles=True, preserve_case=False)

# definindo stopwords
mystopwords = set(stopwords.words('english'))

# criando função para remover stopwords e caracteres numericos
def remove_stopwords_num(tokens):
    return [token for token in tokens if token not in mystopwords and not token.isdigit()]

# criando uma função para o pre-processamento
def preprocess(texts):
    return [remove_stopwords_num(tweeter.tokenize(content)) for content in texts]

# passando as funções nos posts
my_text = preprocess(df_amostra['content'])

# separando as categorias dos sentimentos
my_categ = df_amostra['sentiment']

# checando as novas dimensões
print(len(my_text), len(my_categ))

16759 16759


In [8]:
# checando os primeiros elementos da nova lista dos textos
my_text[0:5]

[['want', 'trade', 'someone', 'houston', 'tickets', ',', 'one', '.'],
 ['re-pinging', ':', 'go', 'prom', '?', 'bc', 'bf', 'like', 'friends'],
 ['hmmm', '.', 'http://www.djhero.com/'],
 ['cant', 'fall', 'asleep'],
 ['choked', 'retainers']]

## Machine Learning

In [9]:
# separando os dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(my_text, my_categ, random_state=1234)
print(len(X_train), len(X_val), len(y_train), len(y_val))

12569 4190 12569 4190


In [10]:
# preparando os dados de treino no formato Doc2Vec
train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(X_train)]

# visualizando os primeiros elementos
train_doc2vec[0:5]

[TaggedDocument(words=["caaaaan't", 'sleep', '...', '3.30', '!', 'wahhhh', '...', 'wanna', 'cry'], tags=['0']),
 TaggedDocument(words=['based', 'future', 'forgetting', '/', 'ignoring', 'present', ',', 'best', 'keeper', 'according', 'dhoni', 'parthiv'], tags=['1']),
 TaggedDocument(words=['good', 'morning', '!', 'early', 'bad', 'conscience', ',', 'trying', 'make', 'taking', 'day', 'yesterday', ',', '?', ':p'], tags=['2']),
 TaggedDocument(words=['hahaha', "chivalry's", 'dead', ',', 'rare'], tags=['3']),
 TaggedDocument(words=['joining', 'twitter'], tags=['4'])]

In [11]:
# definindo os parâmetros para o modelo
model = Doc2Vec(vector_size=50, alpha=0.025, min_counts=5, dm=1, epochs=100)

# instanciando o construtor
model.build_vocab(train_doc2vec)

# treinando o modelo, somente com os dados de treino!
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)

# salvando o modelo
model.save('d2v.model')
print("Model saved")

Model saved


In [12]:
# importando o modelo treinado se precisar
model = Doc2Vec.load('d2v.model')

# inferindo o modelo em várias etapas para uma representação estável
train_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in X_train]
test_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in X_val]

# instanciando o Logistic Regression
logreg = LogisticRegression(class_weight="balanced")
logreg.fit(train_vectors, y_train)

# realizando as previsões
y_pred = logreg.predict(test_vectors)

# analisando o classification report
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

   happiness       0.34      0.55      0.42       713
     neutral       0.47      0.53      0.50      1595
       worry       0.61      0.41      0.49      1882

    accuracy                           0.48      4190
   macro avg       0.47      0.49      0.47      4190
weighted avg       0.51      0.48      0.48      4190



## Referência
https://github.com/practical-nlp/practical-nlp/blob/master/Ch4/02_Doc2Vec_Example.ipynb